In [1]:
!pip install git+https://github.com/openai/shap-e.git

  Cloning https://github.com/openai/shap-e.git to /tmp/pip-req-build-4qf8gvxt
  Running command git clone --filter=blob:none --quiet https://github.com/openai/shap-e.git /tmp/pip-req-build-4qf8gvxt
  Resolved https://github.com/openai/shap-e.git to commit 50131012ee11c9d2617f3886c10f000d3c7a3b43
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-v7jhqdbd/clip_abd3ed7dbfca4617bc5eca408b85c395
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-v7jhqdbd/clip_abd3ed7dbfca4617bc5eca408b85c395
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 42.6

In [2]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
from shap_e.util.image_util import load_image

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
xm = load_model('transmitter', device=device)
model = load_model('image300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

/usr/local/lib/python3.10/dist-packages/shap_e/models/nn/checkpoint.py:32: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, run_function, length, *args):
/usr/local/lib/python3.10/dist-packages/shap_e/models/nn/checkpoint.py:44: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, *output_grads):
/usr/local/lib/python3.10/dist-packages/shap_e/models/nn/checkpoint.py:62: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, run_function, length_1, length_2, *args):
/usr/local/lib/python3.10/dist-packages/shap_e/models/nn/checkpoint.py:87: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead

  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

/usr/local/lib/python3.10/dist-packages/shap_e/models/download.py:136: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(path, map_location=device)
100%|██████

  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

/usr/local/lib/python3.10/dist-packages/shap_e/models/download.py:136: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(path, map_location=device)


In [14]:
batch_size = 4
guidance_scale = 3.0

# To get the best result, you should remove the background and show only the object of interest to the model.
image = load_image("/content/20220619_121831.jpg")

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(images=[image] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [11]:
from PIL import Image
import torch



def save_gif(images, save_path="output.gif", duration=100):

    # Convert torch tensors to PIL images if necessary
    pil_images = [Image.fromarray((img.permute(1, 2, 0).cpu().numpy() * 255).astype('uint8')) for img in images]

    # Save images as a GIF
    pil_images[0].save(
        save_path,
        save_all=True,
        append_images=pil_images[1:],
        duration=duration,  # Duration of each frame in milliseconds
        loop=0  # Loop forever
    )
    print(f"GIF saved at {save_path}")


In [10]:
render_mode = 'nerf' # you can change this to 'stf' for mesh rendering
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    # save_gif(images, save_path=f"output_{i}.gif", duration=100)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAN7g4tTY3cbM2b+/1si0ormwqbCyw5Gh0LyWf4+ZyIiXwIeSr4…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAOm9j92vgdioZKyotOGiYcSiWbSkZpuhpKChfuacWdaeTcaVWq…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAN3TxNzOvNrGqsrEura/x9y1hc2thcC1oa2zt6GruLmkiaKprJ…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAN3RstvOrtXKrd7EjtfCmde7g9e4fc66j724qtOwccyxes6nZs…

In [7]:
!pip install Pillow


In [15]:

from PIL import Image

# Function to save a sequence of images as a GIF
def save_gif(images, save_path="output.gif", duration=100):


    if isinstance(images[0], Image.Image):
        images[0].save(
            save_path,
            save_all=True,
            append_images=images[1:],
            duration=duration,
            loop=0
        )
        print(f"GIF saved at {save_path}")
    else:
        print("Error: images should be a list of PIL Image objects.")


for i, latent in enumerate(latents):
    # Generate the images from the latent vector
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)

    # Save the generated GIF
    save_gif(images, save_path=f"output_{i}.gif", duration=100)


GIF saved at output_0.gif
GIF saved at output_1.gif
GIF saved at output_2.gif
GIF saved at output_3.gif
